In [2]:
#load the necessary libraries for reading the documents
import pandas as pd
import numpy as np
import logging

In [3]:
#we use the standard IMDB dataset that consist of 100,000 reviews of movies. The dataset consist of 25k observations labeled,
#(as positive and negative), 25k test reviews and 50k unlabeled reviews.
#read the data sources
labeled=pd.read_csv('/home/ubuntu/git/word2vec_exp/labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)
unlabeled=pd.read_csv('/home/ubuntu/git/word2vec_exp/unlabeledTrainData.tsv.zip', header=0, delimiter="\t", quoting=3)
testdata=pd.read_csv('/home/ubuntu/git/word2vec_exp/testData.tsv.zip', header=0, delimiter="\t", quoting=3)

In [4]:
#explore the dataset
print(labeled.head(2), len(labeled))
print(unlabeled.head(2), len(labeled))
print(testdata.head(2), len(labeled))

         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ... 25000
          id                                             review
0   "9999_0"  "Watching Time Chasers, it obvious that it was...
1  "45057_0"  "I saw this film about 20 years ago and rememb... 25000
           id                                             review
0  "12311_10"  "Naturally in a film who's main themes are of ...
1    "8348_2"  "This movie is a disaster within a disaster fi... 25000


In [5]:
#we make sure there are no repeated observations, we use the command assert to verify that we have as many unique id's as 
assert labeled.id.nunique() == labeled.review.nunique(), "The number of unique reviews is different to unique id's"

AssertionError: The number of unique reviews is different to unique id's

In [10]:
#we repeat the same procedure for the test dataset and for the unlabeled data
assert unlabeled.id.nunique() == unlabeled.review.nunique(), "The number of unique reviews is different to unique id's"

AssertionError: The number of unique reviews is different to unique id's

In [11]:
assert testdata.id.nunique() == testdata.review.nunique(), "The number of unique reviews is different to unique id's"

AssertionError: The number of unique reviews is different to unique id's

In [6]:
#we do a quick group by to find where error resides for the labeled dataset
labeled.groupby('review').count().sort_values('id', ascending=False)

,id,sentiment
review,,
"""How has this piece of crap stayed on TV this long? It's terrible. It makes me want to shoot someone. It's so fake that it is actually worse than a 1940s sci-fi movie. I'd rather have a stroke than watch this nonsense. I remember watching it when it first came out. I thought, hey this could be interesting, then I found out how absolutely, insanely, ridiculously stupid it really was. It was so bad that I actually took out my pocket knife and stuck my hand to the table.<br /><br />Please people, stop watching this and all other reality shows, they're the trash that is jamming the networks and canceling quality programming that requires some thought to create.""",3,3
"""This show comes up with interesting locations as fast as the travel channel. It is billed as reality but in actuality it is pure prime time soap opera. It's tries to use exotic locales as a facade to bring people into a phony contest & then proceeds to hook viewers on the contestants soap opera style.<br /><br />It also borrows from an early CBS game show pioneer- Beat The Clock- by inventing situations for its contestants to try & overcome. Then it rewards the winner money. If they can spice it up with a little interaction between the characters, even better. While the game format is in slow motion versus Beat The Clock- the real accomplishment of this series is to escape reality. <br /><br />This show has elements of several types of successful past programs. Reality television, hardly, but if your hooked on the contestants, locale or contest, this is your cup of tea. If your not, this entire series is as I say, drivel dripping with gravy. It is another show hiding behind the reality label which is the trend it started in 2000.<br /><br />It is slick & well produced, so it might last a while yet. After all, so do re-runs of Gilligan's Island, Green Acres, The Beverly Hillbillies & The Brady Bunch. This just doesn't employ professional actors. The intelligence level is about the same.""",3,3
"""When i got this movie free from my job, along with three other similar movies.. I watched then with very low expectations. Now this movie isn't bad per se. You get what you pay for. It is a tale of love, betrayal, lies, sex, scandal, everything you want in a movie. Definitely not a Hollywood blockbuster, but for cheap thrills it is not that bad. I would probably never watch this movie again. In a nutshell this is the kind of movie that you would see either very late at night on a local television station that is just wanting to take up some time, or you would see it on a Sunday afternoon on a local television station that is trying to take up some time. Despite the bad acting, cliché lines, and sub par camera work. I didn't have the desire to turn off the movie and pretend like it never popped into my DVD player. The story has been done many times in many movies. This one is no different, no better, no worse. <br /><br />Just your average movie.""",3,3
"""You do realize that you've been watching the EXACT SAME SHOW for eight years, right? I could understand the initial curiosity of seeing strangers co-exist on an Island, but you'd think that after watching unkempt, stink-ladened heroes run roughshod through the bush with an egg on a spoon for half a decade would be enough to get you to commit to something a little more original (and interesting).<br /><br />And I'm not even speaking of the shows validity which for the record I find questionable. It's just hard to suspend disbelief for \""Bushy Bill\"" eating a rat when the entire crew of producers and camera people are housed in an air conditioned make-shift bio-dome sipping frosty mochcinno's with moxy.<br /><br />What's the appeal here? I don't care about these people or their meandering lives. I just don't get it. But if you DO find yourself being captivated by hairy, unwashed people, I suggest you turn off your TV and just take a trip to your local bus station where you can see people like this

In [13]:
#we filter the repeated observations and see how many observations we lost
#we create a non repeated dataframe
print(len(labeled))
labeled=labeled.drop_duplicates('review')
print(len(labeled))

24904
24904


In [14]:
#repeat the same process with the unlabeled data. We leave the test dataset untouched for now
unlabeled.head()
print (len(unlabeled))
unlabeled=unlabeled.drop_duplicates('review')
print (len(unlabeled))

50000
49507


In [49]:
#Before we implement the Doc2vec we need to clean the text, stem it, remove special characters as well as numeric ones. We will
#use a combination of Gensim, regular expressions and NLTK. We will define two functions to clean the data, one using 
#the preinstalled Gensim pipeline for text cleaning and one defined by us. We will compare the performance of both functions
#for the embeddings, clustering and the overall classification.
#clean the text
from pathos.multiprocessing import ProcessingPool as Pool
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_non_alphanum


# we can do this whole thing across a number of processors 
# initialize pool of processes
p = Pool(ncpus = 4)

def clean_text(text):
    # filter ords greater than 120
    wl = filter(lambda x: ord(x) < 128, text)

    # pass through pre-processing filter
    wl = preprocess_string(text, filters = [stem_text, strip_numeric, 
                                          strip_punctuation, remove_stopwords, 
                                          strip_short, strip_non_alphanum])
    return wl
                    

def clean_textv2(text,remove_stopwords=True):
    import re
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    from bs4 import BeautifulSoup
    #remove special character of beatiful soup
    review_text = re.sub("(<br\s\/>)"," ", text)

    # Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default). For word2vec we don't remove words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #5. Lemmatize
    wnl = WordNetLemmatizer()
    words = list(map(lambda x: wnl.lemmatize(x), words))
    # 5. Return a list of words
    return(words)

In [54]:
#clean the labeled text data and the unlabeled text data using the gensim pipeline
clean_text_labeled= p.map(clean_text, labeled['review'].values.tolist())
clean_text_unlabeled= p.map(clean_text, unlabeled['review'].values.tolist())
#clean_text2a=p.map(clean_text, unlabeled['review'].values.tolist())

In [55]:
#repeat usingt the nltk function
clean_text_labeled_v2= p.map(clean_textv2, labeled['review'].values.tolist())
clean_text_unlabeled_v2= p.map(clean_textv2, unlabeled['review'].values.tolist())

In [56]:
#make sure the functions work as intended. We explore the results of cleaning the data with gensin and our defined 
#text fuinction
print(clean_text_labeled[2])
print(clean_text_labeled_v2[2])

['film', 'start', 'manag', 'nichola', 'bell', 'welcom', 'investor', 'robert', 'carradine', 'primal', 'park', 'secret', 'project', 'mutat', 'primal', 'anim', 'fossil', 'dna', 'like', 'jurassik', 'park', 'scientist', 'resurrect', 'nature', 'fearsom', 'predators', 'sabretooth', 'tiger', 'smilodon', 'scientif', 'ambit', 'turn', 'deadly', 'high', 'voltag', 'fenc', 'open', 'creatur', 'escap', 'begin', 'savag', 'stalk', 'prei', 'human', 'visitor', 'tourist', 'scientific', 'meanwhil', 'youngster', 'enter', 'restrict', 'area', 'secur', 'center', 'attack', 'pack', 'larg', 'pre', 'histor', 'anim', 'deadlier', 'bigger', 'addit', 'secur', 'agent', 'staci', 'haiduk', 'mate', 'brian', 'wimmer', 'fight', 'hardli', 'carnivor', 'smilodons', 'sabretooths', 'themselv', 'course', 'real', 'star', 'star', 'thei', 'astound', 'terrifyingli', 'convincing', 'giant', 'anim', 'savag', 'stalk', 'prei', 'group', 'run', 'afoul', 'fight', 'nature', 'fearsom', 'predators', 'furthermor', 'sabretooth', 'danger', 'slow', 

In [58]:
#verify the lenght of both lists
print (len(clean_text_labeled), len(clean_text_unlabeled))
print (len(clean_text_labeled_v2), len(clean_text_unlabeled_v2))

24904 49507
24904 49507


In [60]:
#we now try to merge the two data sources
clean_text_complete=clean_text_labeled+clean_text_unlabeled
print (len(clean_text_complete))

74411


In [65]:
assert clean_text_labeled[0]==clean_text_complete[0], "not same index position"

In [67]:
#create a list of the ids for both labeld and unlabeled
labeled_ids=labeled['id'].values.tolist()
unlabeled_ids=unlabeled['id'].values.tolist()

In [72]:
#check the lenght of the lists containing that information, we only do this once since we don't need to repeat that operation
#for the other list of text
print ('lenght of labeled ids list is', len(labeled_ids))
print ('lenght of unlabeled ids list is', len(unlabeled_ids))
#merge both lists
complete_ids=labeled_ids+unlabeled_ids
print ('complete_ids', len(complete_ids))

lenght of labeled ids list is 24904
lenght of unlabeled ids list is 49507
complete_ids 74411


In [29]:
#we explplore one observation
clean_text[1:3]
#we need to flat the list


[['classic',
  'war',
  'worlds',
  'timothi',
  'hine',
  'veri',
  'entertain',
  'film',
  'obvious',
  'goe',
  'great',
  'effort',
  'length',
  'faithfulli',
  'recreat',
  'wells',
  'classic',
  'book',
  'hine',
  'succe',
  'watch',
  'film',
  'appreci',
  'fact',
  'standard',
  'predict',
  'hollywood',
  'fare',
  'come',
  'everi',
  'year',
  'spielberg',
  'version',
  'tom',
  'cruis',
  'onli',
  'slightest',
  'resembl',
  'book',
  'obviously',
  'everyon',
  'look',
  'differ',
  'thing',
  'movie',
  'envis',
  'themselv',
  'amateur',
  'critics',
  'look',
  'onli',
  'critic',
  'everyth',
  'thei',
  'rate',
  'movi',
  'import',
  'bases',
  'lik',
  'entertained',
  'peopl',
  'agre',
  'critics',
  'enjoi',
  'effort',
  'hine',
  'faith',
  'wells',
  'classic',
  'novel',
  'veri',
  'entertaining',
  'thi',
  'easi',
  'overlook',
  'critics',
  'perceiv',
  'shortcomings'],
 ['film',
  'start',
  'manag',
  'nichola',
  'bell',
  'welcom',
  'investor

In [30]:
from gensim.models import phrases

def map_phrases(text_stream):

    phrases2 = phrases.Phrases(text_stream, min_count = 10, threshold = 20)

    bigram = phrases.Phraser(phrases2)

    text_stream = list(map(lambda x: bigram[x], text_stream))
    
    return text_stream

clean_text2 = map_phrases(clean_text)

In [31]:
#explore the results of the bigram
print(len(clean_text2))
clean_text2[1]

74411


['classic',
 'war_worlds',
 'timothi_hine',
 'veri',
 'entertain',
 'film',
 'obvious',
 'goe',
 'great',
 'effort',
 'length',
 'faithfulli',
 'recreat',
 'wells',
 'classic',
 'book',
 'hine',
 'succe',
 'watch',
 'film',
 'appreci',
 'fact',
 'standard',
 'predict',
 'hollywood',
 'fare',
 'come',
 'everi',
 'year',
 'spielberg',
 'version',
 'tom_cruis',
 'onli',
 'slightest',
 'resembl',
 'book',
 'obviously',
 'everyon',
 'look',
 'differ',
 'thing',
 'movie',
 'envis',
 'themselv',
 'amateur',
 'critics',
 'look',
 'onli',
 'critic',
 'everyth',
 'thei',
 'rate',
 'movi',
 'import',
 'bases',
 'lik',
 'entertained',
 'peopl',
 'agre',
 'critics',
 'enjoi',
 'effort',
 'hine',
 'faith',
 'wells',
 'classic',
 'novel',
 'veri',
 'entertaining',
 'thi',
 'easi',
 'overlook',
 'critics',
 'perceiv',
 'shortcomings']

In [33]:
# create named tuple keeping the doc labels with their raw text
from collections import namedtuple

doc = namedtuple('doc', 'label content')
part1=labeled['id'].values.tolist()
part2=unlabeled['id'].values.tolist()
labels=part1+part2
ntup_list = []
for index, line in enumerate(clean_text2):
    ntup_list.append(doc(label = labels[index], content = line))

In [34]:
#explore the content
ntup_list[2]

doc(label='"7759_3"', content=['film', 'start', 'manag', 'nichola', 'bell', 'welcom', 'investor', 'robert', 'carradine', 'primal', 'park', 'secret', 'project', 'mutat', 'primal', 'anim', 'fossil', 'dna', 'like', 'jurassik', 'park', 'scientist', 'resurrect', 'nature', 'fearsom', 'predators', 'sabretooth', 'tiger', 'smilodon', 'scientif', 'ambit', 'turn', 'deadly', 'high_voltag', 'fenc', 'open', 'creatur', 'escap', 'begin', 'savag', 'stalk', 'prei', 'human', 'visitor', 'tourist', 'scientific', 'meanwhil', 'youngster', 'enter', 'restrict', 'area', 'secur', 'center', 'attack', 'pack', 'larg', 'pre', 'histor', 'anim', 'deadlier', 'bigger', 'addit', 'secur', 'agent', 'staci', 'haiduk', 'mate', 'brian', 'wimmer', 'fight', 'hardli', 'carnivor', 'smilodons', 'sabretooths', 'themselv', 'course', 'real', 'star', 'star', 'thei', 'astound', 'terrifyingli', 'convincing', 'giant', 'anim', 'savag', 'stalk', 'prei', 'group', 'run_afoul', 'fight', 'nature', 'fearsom', 'predators', 'furthermor', 'sabreto

In [35]:
# helper class to load in the documents and their labels and return the output expected by 
# doc2vec
import logging
from gensim.models import Doc2Vec
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from random import shuffle

class LabeledLineSentence(object):
    def __init__(self, ntup_list_model = [], verbose = True):
        """
        Initialize document iterator
        
        Keyword Arguments: 
        
        fpath -- file path to document
        
        subset -- list of subset labels to filter against
        
        verbose -- Boolean, to log the results to a text file containing progress
        """
        
        self.ntup_list_model = ntup_list_model
        
        self.sentences = []
        
        if verbose:
            logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                                level=logging.INFO)
                  
    
    def __iter__(self):
        """
        yield labeldsentence for each doc
        """
        logging.info('[+] Iterating over documents')
        
        
        # for filenum, f in enumerate(self.allfiles):
            #if filenum == 20:
            #    break
            
        for index, line in enumerate(self.ntup_list_model):
            
            label = line.label
            
            out_content = line.content
            
            sent = LabeledSentence(out_content, [label])
            
            self.sentences.append(sent)
            
            yield sent
                    
    
    def shuffle_sent(self):
        """
        shuffle sentences (has been proven to increase performance of doc2vec)
        """
        
        logging.info('[+] Shuffling sentences....')
            
        shuffle(self.sentences)
        
        return self.sentences

In [37]:
# now for the fun part, time to train the model!
# model parameter informaiotn: https://radimrehurek.com/gensim/models/doc2vec.html
# model source code: https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/doc2vec.py
# doc2vec research paper: https://cs.stanford.edu/~quocle/paragraph_vector.pdf
sentences = LabeledLineSentence(ntup_list_model = ntup_list, verbose = True)
model = Doc2Vec(min_count=10, window = 10, size = 300, sample=1e-5, negative=5, workers=10, 
                alpha = 0.025, min_alpha = 0.025)
model.build_vocab(sentences)
for epoch in range(10):
    model.train(sentences.shuffle_sent(),total_examples=len(labels),epochs=1)
    model.min_alpha = model.alpha

In [38]:
#save model
model_name = "/home/ubuntu/git/word2vec_exp/doc2vec_v3"
model.save(model_name)

In [39]:
import logging
from gensim.models import Doc2Vec
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from random import shuffle
#load model
new_model = Doc2Vec.load("/home/ubuntu/git/word2vec_exp/doc2vec_v3")

In [40]:
# pull out sections
#sections = new_model.docvecs.doctags.keys()
print (len(new_model.docvecs[0]))

300


In [41]:
#vecs
new_model.docvecs

In [54]:
new_model.most_similar('comedy')

[('comedi', 0.9197102189064026),
 ('laughs', 0.8808180093765259),
 ('funny', 0.870806097984314),
 ('laugh_loud', 0.8617198467254639),
 ('comedies', 0.8538097143173218),
 ('funni', 0.8471378087997437),
 ('humor', 0.8407548666000366),
 ('funnier', 0.8405470848083496),
 ('jokes', 0.8387101888656616),
 ('farc', 0.8294789791107178)]

In [55]:
#Transforn the vector into a dataframe corresponding to the documents
reviews = new_model.docvecs.doctags.keys()

all_reviews = list(map(lambda x: new_model.docvecs[x], reviews))
type(all_reviews)
#
#all_dm = list(map(lambda x: new_model.docvecs[x], new_model.docvecs.doctags.keys()))
#all_dm=new_model.docvecs
#labeled_vectors=pd.DataFrame(new_model.docvecs)
#for
#labeled['doc_vec'] = all_dm
labeled_vectors=pd.DataFrame(all_reviews)

In [56]:
#add the label
id_vector=[]
for i in reviews:
    id_vector.append(i)

labeled_vectors['id']=id_vector
labeled_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,id
0,-0.091038,0.018808,-0.076388,0.031891,-0.010208,0.070599,0.024127,0.087209,-0.032185,-0.018227,...,0.001532,0.055050,0.007961,-0.043922,0.029147,0.064303,-0.030690,0.015429,-0.003429,"""5814_8"""
1,-0.099248,0.101052,-0.069167,0.025337,0.014936,0.033777,0.004847,0.098134,-0.173723,-0.102504,...,-0.094643,0.109627,-0.036490,0.078629,-0.003796,-0.044804,0.023470,-0.020505,-0.004396,"""2381_9"""
2,-0.193094,0.178471,0.010350,0.077503,0.045248,-0.125699,-0.038614,-0.100993,0.009445,0.047738,...,-0.025463,-0.009273,0.132779,-0.049568,0.008295,-0.022033,-0.046609,0.115340,-0.007502,"""7759_3"""
3,0.052576,-0.059288,-0.026568,0.073663,0.071514,-0.039055,-0.086349,0.003423,-0.220795,-0.085408,...,-0.210163,0.175158,-0.126672,0.170346,-0.120832,0.012175,-0.021121,-0.051494,0.003745,"""3630_4"""
4,-0.186300,0.124205,-0.113075,0.022893,0.102089,-0.109287,-0.055027,0.016929,0.017367,-0.007997,...,-0.119963,0.135223,0.151406,-0.020490,-0.036162,-0.051358,0.000526,0.133301,-0.007525,"""9495_8"""


In [57]:
#save dataframe
labeled_vectors.to_csv('/home/ubuntu/git/word2vec_exp/labeled_unlabeled_data_doc2vec_v2.csv', index=False)

In [58]:
len(labeled_vectors)

74411

In [13]:
#verify it worked
print(np.shape(labeled['doc_vec'].iloc[10]))
labeled.head()

(300,)


,id,sentiment,review,doc_vec
0,"""5814_8""",1,"""With all this stuff going down at the moment ...","[0.00941616, 0.0680217, -0.0938441, -0.0256403..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...","[-0.0313071, 0.0222344, -0.0382599, -0.0230157..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...","[0.0325313, 0.000933911, -0.124349, -0.0851798..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi...","[-0.0285973, 0.00637894, -0.0512476, -0.065585..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...","[0.0185921, 0.0436636, -0.132684, -0.0573546, ..."


In [17]:
#save to cvs
labeled.to_csv('/home/ubuntu/git/word2vec_exp/labeled_data_doc2vec.csv')

In [16]:
labeled.head()

,id,sentiment,review,doc_vec
0,"""5814_8""",1,"""With all this stuff going down at the moment ...","[0.00941616, 0.0680217, -0.0938441, -0.0256403..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...","[-0.0313071, 0.0222344, -0.0382599, -0.0230157..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...","[0.0325313, 0.000933911, -0.124349, -0.0851798..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi...","[-0.0285973, 0.00637894, -0.0512476, -0.065585..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...","[0.0185921, 0.0436636, -0.132684, -0.0573546, ..."


In [50]:
#save dataframe as 
labeled.to
import tsnet

ModuleNotFoundError: No module named 'tsnet'

In [38]:
for i in new_model.docvecs.doctags.keys():
    print (i)

"5814_8"
"2381_9"
"7759_3"
"3630_4"
"9495_8"
"8196_8"
"7166_2"
"10633_1"
"319_1"
"8713_10"
"2486_3"
"6811_10"
"11744_9"
"7369_1"
"12081_1"
"3561_4"
"4489_1"
"3951_2"
"3304_10"
"9352_10"
"3374_7"
"10782_7"
"5414_10"
"10492_1"
"3350_3"
"6581_7"
"2203_3"
"689_1"
"9152_1"
"6077_1"
"4656_4"
"9727_7"
"1297_8"
"5586_8"
"1119_1"
"11241_1"
"4005_10"
"6827_4"
"9011_9"
"11885_1"
"7897_8"
"4613_4"
"12106_10"
"11427_1"
"10135_2"
"4221_8"
"9983_3"
"9788_9"
"7316_1"
"9897_10"
"9654_10"
"2395_7"
"4439_4"
"8516_2"
"9057_3"
"11289_4"
"301_10"
"4155_10"
"8880_3"
"9569_9"
"1510_8"
"1782_4"
"8352_1"
"5983_8"
"11232_1"
"796_3"
"1150_10"
"11268_1"
"10064_10"
"6666_4"
"11471_7"
"9343_1"
"1142_3"
"6403_3"
"5440_2"
"8697_7"
"8459_3"
"6520_7"
"9952_8"
"8605_10"
"10531_10"
"11736_9"
"2614_1"
"2382_1"
"7168_10"
"9060_1"
"1777_10"
"5466_1"
"10533_10"
"5198_3"
"4784_2"
"11761_1"
"292_10"
"5240_1"
"3088_8"
"4032_4"
"2399_7"
"11259_1"
"8437_3"
"1864_1"
"5107_1"
"8157_4"
"2681_3"
"11724_8"
"12397_8"
"4680_10"
"1889_10"

"7699_10"
"11048_7"
"3123_10"
"3940_1"
"10967_4"
"8658_1"
"9987_9"
"9134_7"
"5653_2"
"4227_1"
"1737_4"
"7489_7"
"1002_7"
"7251_10"
"1004_7"
"10716_3"
"8349_10"
"3121_4"
"8288_7"
"4261_4"
"6290_7"
"4152_10"
"2783_8"
"8833_9"
"157_9"
"8644_1"
"9213_3"
"7472_9"
"46_4"
"2379_8"
"40_8"
"4658_8"
"5068_4"
"1218_8"
"11795_1"
"1563_10"
"7600_1"
"12260_10"
"11819_3"
"11418_4"
"10670_10"
"10306_4"
"1041_1"
"5596_10"
"929_1"
"4792_3"
"1348_3"
"10455_10"
"5746_9"
"11968_10"
"7742_3"
"8328_1"
"1617_4"
"5743_10"
"4951_8"
"4007_4"
"1949_1"
"11858_2"
"3798_2"
"9743_8"
"10814_2"
"4566_1"
"4198_7"
"4001_8"
"4201_9"
"12190_4"
"6111_10"
"10000_8"
"1733_7"
"9448_1"
"10378_2"
"11785_10"
"2370_4"
"10161_9"
"3073_8"
"1931_1"
"7024_2"
"6906_8"
"8617_1"
"1970_4"
"9520_4"
"3023_7"
"8104_7"
"11916_3"
"1485_2"
"11849_4"
"9820_10"
"7464_3"
"4207_4"
"9996_9"
"10657_3"
"3091_10"
"3773_1"
"12230_9"
"12228_8"
"8550_3"
"975_4"
"5571_10"
"7_7"
"1102_8"
"9950_8"
"9183_10"
"3598_3"
"3036_2"
"2522_8"
"10446_2"
"9040_9"
"1126

"8062_2"
"11600_3"
"1425_7"
"5155_9"
"5744_7"
"6713_2"
"10272_10"
"10909_1"
"12284_7"
"3837_4"
"74_8"
"7080_10"
"3107_3"
"4980_8"
"5115_1"
"12375_4"
"10157_4"
"3581_9"
"4873_1"
"4429_7"
"10510_7"
"1997_9"
"10008_7"
"5879_1"
"2045_9"
"1045_2"
"11539_1"
"4448_4"
"2786_7"
"11523_7"
"9022_10"
"194_1"
"4074_2"
"8813_9"
"2802_1"
"12353_9"
"9503_4"
"2054_4"
"7423_4"
"2061_1"
"2261_10"
"11491_8"
"4544_10"
"10359_1"
"11605_10"
"1558_10"
"121_10"
"493_4"
"9573_8"
"10441_10"
"7336_10"
"2120_2"
"88_9"
"7861_10"
"2389_2"
"598_4"
"8165_8"
"829_1"
"1185_8"
"5340_3"
"7475_2"
"8529_9"
"4339_2"
"9941_2"
"1178_10"
"2497_8"
"1690_10"
"10060_9"
"6570_3"
"9898_1"
"5905_9"
"2726_9"
"5108_1"
"2509_4"
"6612_1"
"1224_1"
"5757_1"
"11992_4"
"8438_3"
"10029_1"
"3428_4"
"908_1"
"2605_7"
"11787_9"
"7716_9"
"8159_1"
"8556_1"
"12455_4"
"6159_1"
"3365_1"
"10463_1"
"4629_1"
"9918_10"
"10887_7"
"9214_7"
"12183_4"
"3647_4"
"6110_8"
"2450_3"
"251_10"
"1566_3"
"4333_2"
"10221_8"
"515_1"
"615_10"
"6861_8"
"11709_3"
"4199_7"


"1532_10"
"3962_10"
"6699_8"
"7241_2"
"8646_7"
"2967_9"
"6385_10"
"549_1"
"2380_9"
"1524_7"
"6428_8"
"3100_1"
"1693_10"
"7169_8"
"7392_8"
"6172_4"
"7645_7"
"11162_10"
"5602_3"
"11371_1"
"664_7"
"4475_8"
"5481_1"
"10223_10"
"6358_10"
"9611_8"
"9266_1"
"4491_10"
"12249_2"
"7123_2"
"9567_7"
"11488_10"
"10133_1"
"8789_1"
"8267_1"
"4411_4"
"12381_1"
"8179_7"
"11002_8"
"9874_1"
"6362_4"
"10088_1"
"5330_7"
"7628_1"
"1250_4"
"12437_7"
"3634_8"
"11809_10"
"3869_9"
"6188_1"
"10447_1"
"8992_9"
"5114_10"
"5815_10"
"10622_10"
"4249_10"
"9862_9"
"1075_10"
"3798_9"
"2915_7"
"7427_9"
"6655_7"
"123_1"
"9130_4"
"3917_9"
"9183_1"
"10782_1"
"2240_7"
"3219_2"
"8890_1"
"3227_7"
"7622_4"
"2807_3"
"3264_8"
"10072_9"
"1681_7"
"8213_9"
"10785_1"
"11060_10"
"8698_3"
"6153_7"
"4220_7"
"47_2"
"3927_9"
"5495_8"
"2480_2"
"1949_8"
"6363_1"
"5808_10"
"11707_1"
"10827_10"
"9736_8"
"7985_4"
"3954_9"
"4453_3"
"2543_10"
"5186_3"
"10893_3"
"3242_8"
"7964_4"
"12092_7"
"5422_9"
"7006_8"
"12443_9"
"4337_1"
"2711_7"
"5011_10"


"8366_4"
"12065_7"
"3645_3"
"851_7"
"10080_2"
"7474_1"
"8424_9"
"4002_1"
"9383_8"
"11129_2"
"9231_8"
"9621_4"
"1715_3"
"4995_3"
"7015_8"
"7839_4"
"11346_2"
"7709_1"
"4561_10"
"1539_3"
"6238_9"
"10822_10"
"4559_10"
"7092_8"
"1570_2"
"4205_1"
"11397_10"
"367_10"
"562_8"
"10197_1"
"2865_1"
"11909_7"
"5825_1"
"7003_4"
"3442_2"
"12176_8"
"11807_2"
"1071_4"
"813_3"
"9494_1"
"12115_4"
"11017_9"
"3116_10"
"10055_3"
"11840_10"
"10740_8"
"6205_1"
"8343_7"
"4522_9"
"1575_1"
"12023_9"
"10054_10"
"9721_7"
"6937_8"
"5821_4"
"11354_10"
"11792_1"
"4931_1"
"6025_9"
"8280_8"
"9565_3"
"4277_3"
"9216_10"
"8582_9"
"3001_3"
"4932_4"
"1677_9"
"0_3"
"4465_7"
"1899_7"
"7626_3"
"9093_10"
"11028_4"
"11490_9"
"8521_3"
"867_1"
"1301_4"
"10154_8"
"12359_1"
"7941_8"
"8728_7"
"1941_9"
"4591_9"
"2079_4"
"4661_10"
"9227_9"
"2736_9"
"10174_4"
"2712_10"
"8788_3"
"10907_9"
"819_10"
"11262_3"
"4543_1"
"3440_8"
"9796_1"
"220_4"
"8298_1"
"6781_4"
"11146_4"
"4281_1"
"5941_8"
"765_9"
"9452_1"
"10343_7"
"10179_9"
"8532_7"
"7205

...

"7829_7"
"3053_8"
"10507_1"
"5968_10"
"4413_8"
"1062_4"
"11208_8"
"8757_2"
"3878_10"
"7575_4"
"10860_7"
"9870_4"
"8505_8"
"5536_1"
"932_3"
"6018_1"
"2001_9"
"319_9"
"2171_1"
"7167_3"
"226_4"
"12296_8"
"9207_10"
"8527_8"
"8354_8"
"1936_1"
"9425_10"
"11436_1"
"11255_1"
"7130_10"
"10339_7"
"2695_3"
"2928_10"
"10465_8"
"9704_1"
"4904_3"
"5853_3"
"3102_1"
"4463_1"
"4490_1"
"1732_10"
"10560_9"
"1142_10"
"4634_10"
"4843_7"
"4430_1"
"1763_9"
"4762_10"
"6351_3"
"5730_1"
"1395_3"
"7938_2"
"4882_1"
"2877_1"
"7131_1"
"1140_3"
"5245_4"
"6404_8"
"3510_7"
"4309_3"
"1440_7"
"6957_7"
"1067_7"
"3136_8"
"46_9"
"3885_10"
"4815_10"
"5440_10"
"3781_1"
"8313_2"
"6258_8"
"7735_10"
"4169_2"
"324_8"
"8643_7"
"11298_3"
"8187_9"
"3464_1"
"9312_1"
"11004_8"
"6350_2"
"12000_8"
"5419_1"
"6312_7"
"9251_3"
"1998_9"
"2014_4"
"7457_4"
"7186_2"
"1155_10"
"5472_2"
"9323_2"
"3523_9"
"11487_10"
"5603_7"
"2678_1"
"8296_8"
"11136_1"
"507_4"
"381_1"
"8604_10"
"5928_8"
"11832_4"
"12460_1"
"12499_2"
"2706_2"
"7322_10"
"9700_1"
"

"8198_7"
"9044_9"
"4172_4"
"5237_1"
"8391_8"
"75_8"
"682_3"
"10840_9"
"3871_8"
"10801_8"
"8369_1"
"8040_9"
"2195_10"
"522_1"
"5942_7"
"10873_4"
"12199_9"
"11440_10"
"10112_1"
"9438_10"
"3062_10"
"5781_10"
"4173_4"
"1487_1"
"4639_3"
"8012_1"
"4700_9"
"1856_9"
"4705_1"
"10941_1"
"2086_3"
"12340_4"
"8248_9"
"2798_8"
"11650_9"
"5029_8"
"7215_1"
"124_10"
"10630_8"
"4493_1"
"2002_1"
"11875_4"
"4527_9"
"11510_10"
"8413_8"
"9860_7"
"10526_9"
"9039_4"
"6468_7"
"514_1"
"161_8"
"11130_3"
"11483_2"
"2316_10"
"3031_8"
"3990_8"
"712_9"
"11646_4"
"6799_9"
"400_2"
"7612_9"
"4660_7"
"2211_3"
"6302_2"
"847_3"
"4273_10"
"7703_8"
"2337_10"
"4568_10"
"5531_10"
"2759_1"
"1720_10"
"7469_3"
"9933_8"
"8058_8"
"2027_4"
"2033_8"
"358_10"
"3373_7"
"3668_1"
"3269_8"
"1188_8"
"8466_1"
"8689_3"
"1867_2"
"8072_1"
"7038_2"
"4755_7"
"2546_1"
"4221_1"
"6304_1"
"8572_7"
"3126_1"
"5_3"
"12247_8"
"1242_9"
"1991_3"
"5930_3"
"8945_1"
"9165_1"
"1858_3"
"4130_9"
"1445_2"
"11520_3"
"1914_4"
"9409_8"
"11496_4"
"8088_3"
"4486_4"


"191_3"
"1451_2"
"5216_8"
"10923_1"
"12032_8"
"4357_10"
"7754_2"
"5274_3"
"4053_8"
"8448_10"
"2054_9"
"9500_7"
"10693_8"
"11694_7"
"2022_4"
"3410_3"
"758_2"
"1793_4"
"1840_10"
"7570_10"
"8172_1"
"2038_7"
"3246_4"
"1963_4"
"6434_3"
"10696_7"
"12018_3"
"8470_2"
"2317_1"
"7101_2"
"10160_4"
"7010_10"
"8522_2"
"324_1"
"930_7"
"7007_8"
"9431_10"
"7773_1"
"9546_8"
"9818_7"
"9463_10"
"4857_7"
"12050_2"
"5036_9"
"3505_1"
"7256_3"
"1922_9"
"5950_4"
"1594_8"
"7938_7"
"11177_3"
"4769_7"
"8645_8"
"4154_10"
"823_4"
"5139_4"
"3366_1"
"2943_1"
"3411_1"
"4383_9"
"4955_9"
"6285_1"
"3533_10"
"6218_7"
"10546_1"
"2714_10"
"11395_4"
"3563_4"
"5925_3"
"8502_1"
"2133_10"
"5943_10"
"7050_3"
"11941_4"
"512_10"
"2201_3"
"10273_2"
"506_2"
"11685_8"
"915_4"
"1153_10"
"9931_4"
"11703_1"
"3046_4"
"2458_2"
"10231_10"
"4077_1"
"49_4"
"9387_9"
"4115_1"
"7056_4"
"1796_2"
"4462_9"
"5628_7"
"3777_4"
"9208_7"
"4895_10"
"1594_1"
"337_9"
"298_4"
"10650_8"
"5785_10"
"7485_8"
"10454_9"
"1850_10"
"4465_2"
"9354_9"
"5461_3"
"190

"9503_7"
"3925_10"
"1091_2"
"1933_8"
"10033_1"
"187_8"
"12443_1"
"8415_7"
"9545_7"
"5296_10"
"11694_4"
"4094_2"
"7933_10"
"5761_4"
"10537_1"
"6439_1"
"4571_1"
"9533_3"
"4329_7"
"11428_7"
"5596_3"
"11796_3"
"6364_8"
"10336_4"
"3764_8"
"9253_10"
"9657_3"
"5557_4"
"8332_9"
"8011_1"
"4280_10"
"8231_10"
"6235_8"
"10502_9"
"12102_4"
"3456_9"
"3680_9"
"2076_1"
"1564_10"
"11539_9"
"6850_3"
"9093_4"
"4689_10"
"6251_1"
"6550_10"
"3251_7"
"5541_2"
"7406_9"
"11342_9"
"6968_3"
"12145_10"
"3000_8"
"1140_10"
"8344_4"
"1589_10"
"2343_10"
"3340_1"
"1690_1"
"576_3"
"2558_1"
"8417_3"
"12360_10"
"5238_7"
"2785_3"
"536_4"
"4700_1"
"3432_4"
"8051_7"
"3762_3"
"5752_8"
"8669_7"
"776_7"
"1543_1"
"6891_3"
"6301_10"
"8435_10"
"11765_8"
"707_8"
"3214_3"
"11292_10"
"3135_9"
"12268_1"
"9507_3"
"8729_2"
"11637_8"
"11206_1"
"7162_1"
"10287_4"
"9571_9"
"10138_8"
"5777_4"
"3747_10"
"5895_4"
"3357_1"
"4006_8"
"8142_2"
"260_7"
"7599_3"
"1855_9"
"10475_8"
"5804_1"
"12361_4"
"3491_8"
"5186_10"
"3617_8"
"2583_10"
"2172_7"
"

"8715_10"
"210_4"
"8199_10"
"2899_1"
"803_10"
"10911_1"
"9821_8"
"6398_8"
"2886_1"
"6719_2"
"11537_4"
"2254_1"
"524_10"
"5631_8"
"12185_3"
"2397_4"
"9658_3"
"6818_10"
"5787_4"
"9785_3"
"12331_8"
"4764_10"
"11679_7"
"1062_10"
"9501_2"
"11032_7"
"9006_4"
"4146_10"
"11744_1"
"365_10"
"4529_8"
"3176_4"
"5975_1"
"10969_10"
"4294_2"
"6644_8"
"12327_1"
"8281_7"
"7293_3"
"2907_7"
"879_2"
"12031_7"
"11921_2"
"4157_8"
"3636_1"
"10167_7"
"9948_8"
"10954_10"
"5122_3"
"533_2"
"9447_1"
"9984_9"
"2784_1"
"9595_10"
"3659_9"
"7850_8"
"6844_10"
"6926_10"
"1156_9"
"4606_3"
"10296_8"
"12492_7"
"2245_4"
"6075_2"
"9036_10"
"10775_8"
"5370_7"
"9586_3"
"12307_10"
"9213_7"
"10241_8"
"4453_10"
"2484_10"
"691_3"
"7636_2"
"3459_4"
"10644_3"
"4587_1"
"8914_8"
"11760_2"
"5509_10"
"8628_4"
"8190_4"
"5747_8"
"5997_9"
"1897_10"
"1328_10"
"8660_1"
"6705_10"
"11140_1"
"1635_7"
"7601_4"
"7741_9"
"1141_10"
"6953_7"
"7827_10"
"12220_8"
"1195_1"
"2243_4"
"2447_10"
"4299_8"
"8683_9"
"9064_4"
"6187_2"
"9263_8"
"10222_4"
"302_

In [67]:
data_vectors=pd.DataFrame(new_model.docvecs.doctag_syn0)
print(len(new_model.docvecs.doctag_syn0))
print (len(data_vectors))
print (len(labeled))

24904
24904
24904


In [73]:
type(new_model.docvecs.doctags)
new=pd.DataFrame(zip(new_model.docvecs.doctags.keys(),new_model.docvecs.doctag_syn0)

dict_keys(['"5814_8"', '"2381_9"', '"7759_3"', '"3630_4"', '"9495_8"', '"8196_8"', '"7166_2"', '"10633_1"', '"319_1"', '"8713_10"', '"2486_3"', '"6811_10"', '"11744_9"', '"7369_1"', '"12081_1"', '"3561_4"', '"4489_1"', '"3951_2"', '"3304_10"', '"9352_10"', '"3374_7"', '"10782_7"', '"5414_10"', '"10492_1"', '"3350_3"', '"6581_7"', '"2203_3"', '"689_1"', '"9152_1"', '"6077_1"', '"4656_4"', '"9727_7"', '"1297_8"', '"5586_8"', '"1119_1"', '"11241_1"', '"4005_10"', '"6827_4"', '"9011_9"', '"11885_1"', '"7897_8"', '"4613_4"', '"12106_10"', '"11427_1"', '"10135_2"', '"4221_8"', '"9983_3"', '"9788_9"', '"7316_1"', '"9897_10"', '"9654_10"', '"2395_7"', '"4439_4"', '"8516_2"', '"9057_3"', '"11289_4"', '"301_10"', '"4155_10"', '"8880_3"', '"9569_9"', '"1510_8"', '"1782_4"', '"8352_1"', '"5983_8"', '"11232_1"', '"796_3"', '"1150_10"', '"11268_1"', '"10064_10"', '"6666_4"', '"11471_7"', '"9343_1"', '"1142_3"', '"6403_3"', '"5440_2"', '"8697_7"', '"8459_3"', '"6520_7"', '"9952_8"', '"8605_10"', '"10

In [69]:
#data_vectors.head()
final=pd.merge(labeled,data_vectors)


MergeError: No common columns to perform merge on

In [15]:
print(len(final))

NameError: name 'final' is not defined

doc_labels = model.docvecs.doctags.keys()
pd.set_option('display.max_colwidth', -1)
model.docvecs.most_similar('"2517_2"') # most similar documents to current label


In [61]:
print (labeled['review'][labeled['id']=='"4584_1"'])
print (labeled['review'][labeled['id']=='"2517_2"'])


4484    "A definite no. A resounding NO. This movie is an absolute dud.<br /><br />Having been recommended to me by a friend very much into \"that sort of thing,\" I watched this movie with some anticipation of being informed, changed, moved, altered, uplifted, and all the other positive mystical things that could happen to me when I suddenly see The Truth. Now this may sound like someone who is already predisposed to poo-pooing anything dealing with the metaphysical, the metaphysical/physical boundaries of existence. Believe me, I am not such a person. I try to be open about any presentation and then decide accordingly.<br /><br />In terms of content, the only thing I found mildly interesting and informative, was the bit about peptides, emotions, addiction, and cellular receptors. That was the only \"unifying\" element I could find in the documentary part of this film. The rest of the documentary rambled around several topics and never seemed to unify and cohere, try to tie up and con

In [41]:
#model.most_similar('"2381_9"')
#model.most_similar(""2381_9"")
#labeled.head()
namedtuple

<function collections.namedtuple>

In [28]:
#we can get the 
np.shape(model.docvecs)

(24904, 300)

In [34]:
m

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
# create named tuple keeping the doc labels with their raw text
from collections import namedtuple

doc = namedtuple('doc', 'label content')

ntup_list = []
for index, line in enumerate(sections_clean2):
    ntup_list.append(doc(label = df['DocNumber'].values.tolist()[index], content = line))